<a href="https://colab.research.google.com/github/frontenddeveloper4web/Chatbot-with-OpenAI-/blob/main/Hybrid_Recommendation_System_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
data = pd.read_csv("fashion_products.csv")
data.head(10)

,User ID,Product ID,Product Name,Brand,Category,Price,Rating,Color,Size
0,19,1,Dress,Adidas,Men's Fashion,40,1.043159,Black,XL
1,97,2,Shoes,H&M,Women's Fashion,82,4.026416,Black,L
2,25,3,Dress,Adidas,Women's Fashion,44,3.337938,Yellow,XL
3,57,4,Shoes,Zara,Men's Fashion,23,1.049523,White,S
4,79,5,T-shirt,Adidas,Men's Fashion,79,4.302773,Black,M
5,98,6,Dress,Adidas,Men's Fashion,47,1.379566,Yellow,L
6,16,7,Jeans,Gucci,Men's Fashion,37,1.356750,White,XL
7,63,8,Sweater,Zara,Kids' Fashion,64,4.360303,Blue,XL
8,96,9,Sweater,H&M,Men's Fashion,53,4.466182,Green,XL
9,36,10,T-shirt,Zara,Kids' Fashion,55,4.093234,White,XL


In [29]:
!pip install surprise

In [30]:
from surprise import Dataset, Reader, SVD # Import with the corrected name
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [31]:
# Content-Based Filtering
content_df = data[['Product ID', 'Product Name', 'Brand',
                   'Category', 'Color', 'Size']]
content_df['Content'] = content_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

# Use TF-IDF vectorizer to convert content into a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer()
# Fit and transform the 'Content' column of content_df
content_matrix = tfidf_vectorizer.fit_transform(content_df['Content'])

content_similarity = linear_kernel(content_matrix, content_matrix)

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data[['User ID',
                                  'Product ID',
                                  'Rating']], reader)

def get_content_based_recommendations(product_id, top_n):
  index = content_df[content_df['Product ID'] == product_id].index[0]
  similarity_scores = content_similarity[index]
  similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
  recommendations = content_df.loc[similar_indices, 'Product ID'].values
  return recommendations

<ipython-input-31-58b29e9c3ce9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content_df['Content'] = content_df.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)


In [32]:
get_content_based_recommendations(5, 10)

array([  1, 221, 989, 331, 570, 817, 890,   6, 423, 720])

In [33]:
# Collaborative Filtering
algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

def get_collaborative_filtering_recommendations(user_id, top_n):
  testset = trainset.build_anti_testset()
  testset = filter(lambda x: x[0] == user_id, testset)
  predictions = algo.test(testset)
  predictions.sort(key=lambda x: x.est, reverse=True)
  recommendations = [prediction.iid for prediction in predictions[:top_n]]
  return recommendations

In [37]:
get_collaborative_filtering_recommendations(79,10)

[534, 26, 581, 562, 692, 426, 672, 108, 858, 269]

In [38]:
# Hybrid Approach
def get_hybrid_recommendations(user_id, product_id, top_n):
  content_based_recommendations = get_content_based_recommendations(product_id, top_n)
  collaborative_filtering_recommendations = get_collaborative_filtering_recommendations(user_id, top_n)
  hybrid_recommendations = list(set(content_based_recommendations + collaborative_filtering_recommendations))
  return hybrid_recommendations[:top_n]

In [39]:
user_id = 79
product_id = 5
top_n = 10
recommendations = get_hybrid_recommendations(user_id, product_id, top_n)

print(f"Recommendations for User {user_id} based on Product {product_id}:")
for i, recommendation in enumerate(recommendations):
  print(f"{i + 1}. Product ID: {recommendation}")

Recommendations for User 79 based on Product 5:
1. Product ID: 1281
2. Product ID: 1570
3. Product ID: 535
4. Product ID: 1262
5. Product ID: 989
6. Product ID: 114
7. Product ID: 247
8. Product ID: 1562
9. Product ID: 1243
10. Product ID: 893
